In [1]:
import numpy as np

In [3]:
A = np.array([[1,2],[3,4]])

In [4]:
A

array([[1, 2],
       [3, 4]])

In [5]:
A**2

array([[ 1,  4],
       [ 9, 16]])

In [9]:
((((A-A.mean(0))**2).sum(1))**(1/2)).mean()

1.4142135623730951

In [10]:
A.mean(0)

array([2., 3.])

In [11]:
A-A.mean(0)

array([[-1., -1.],
       [ 1.,  1.]])

In [12]:
%matplotlib inline
%load_ext autoreload
%autoreload 1

import cv2
import matplotlib.pyplot as plt
import numpy as np

%aimport lab1
from lab1 import *

np.set_printoptions(precision=6)  # Print less digits

In [18]:
src1 = np.array([[0.0, 0.0], [331.0, 0.0], [331.0, 474.0], [0.0, 474.0]], dtype="float32")
dst1 = np.array([[182.0, 94.0], [482.0, 48.0], [598.0, 466.0], [146.0, 533.0]], dtype="float32")


In [19]:
import cv2 

In [26]:
M = cv.getPerspectiveTransform(src1,dst1)

In [27]:
M

array([[ 9.012217e-01, -1.792522e-01,  1.820000e+02],
       [-1.394830e-01,  5.490343e-01,  9.400000e+01],
       [-1.062811e-05, -7.075535e-04,  1.000000e+00]])

In [37]:
src2 = np.array([[434.375, 93.625], [429.625, 190.625], 
                 [533.625, 301.875], [452.375, 460.625], 
                 [558.375, 188.625], [342.444, 362.596], 
                 [345.625, 41.875], [341.625, 146.125]])
dst2 = np.array([[204.780, 92.367], [201.875, 190.625], 
                 [297.125, 296.875], [224.446, 456.556],
                 [318.407, 192.155], [107.625, 371.375],
                 [109.875, 26.624], [106.625, 138.125]])

In [38]:
src2

array([[434.375,  93.625],
       [429.625, 190.625],
       [533.625, 301.875],
       [452.375, 460.625],
       [558.375, 188.625],
       [342.444, 362.596],
       [345.625,  41.875],
       [341.625, 146.125]])

In [42]:
M2 = cv.findHomography(src2,dst2)

In [43]:
M2

(array([[ 1.739240e+00,  9.004454e-03, -4.469697e+02],
        [ 2.754460e-01,  1.514284e+00, -1.216115e+02],
        [ 1.162229e-03,  1.834664e-05,  1.000000e+00]]),
 array([[1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1]], dtype=uint8))

In [49]:
import numpy as N

In [84]:
def Normalization(nd, x):
    '''
    Normalization of coordinates (centroid to the origin and mean distance of sqrt(2 or 3).
    Input
    -----
    nd: number of dimensions, 3 here
    x: the data to be normalized (directions at different columns and points at rows)
    Output
    ------
    Tr: the transformation matrix (translation plus scaling)
    x: the transformed data
    '''

    x = np.asarray(x)
    m = np.mean(x, 0)
    #s = np.std(x)
    s = (((x - m)**2).mean())**(1/2)
    #s = ((((x-m)**2).sum(1))**(1/2)).mean()
    if nd == 2:
        Tr = np.array([[s, 0, m[0]], [0, s, m[1]], [0, 0, 1]])
    else:
        Tr = np.array([[s, 0, 0, m[0]], [0, s, 0, m[1]], [0, 0, s, m[2]], [0, 0, 0, 1]])
        
    Tr = np.linalg.inv(Tr)
    x = np.dot( Tr, np.concatenate( (x.T, np.ones((1,x.shape[0]))) ) )
    x = x[0:nd, :].T
    
    print("test1")

    return Tr, x

In [85]:
def DLTcalib(nd, xyz, uv):
    '''
    Camera calibration by DLT using known object points and their image points.
    This code performs 2D or 3D DLT camera calibration with any number of views (cameras).
    For 3D DLT, at least two views (cameras) are necessary.
    Inputs:
     nd is the number of dimensions of the object space: 3 for 3D DLT and 2 for 2D DLT.
     xyz are the coordinates in the object 3D or 2D space of the calibration points.
     uv are the coordinates in the image 2D space of these calibration points.
     The coordinates (x,y,z and u,v) are given as columns and the different points as rows.
     For the 2D DLT (object planar space), only the first 2 columns (x and y) are used.
     There must be at least 6 calibration points for the 3D DLT and 4 for the 2D DLT.
    Outputs:
     L: array of the 8 or 11 parameters of the calibration matrix
     err: error of the DLT (mean residual of the DLT transformation in units of camera coordinates).
    '''
    
    #Convert all variables to numpy array:
    xyz = N.asarray(xyz)
    uv = N.asarray(uv)
    #number of points:
    np = xyz.shape[0]
    #Check the parameters:
    if uv.shape[0] != np:
        raise ValueError('xyz (%d points) and uv (%d points) have different number of points.' %(np, uv.shape[0]))
    if (nd == 2 and xyz.shape[1] != 2) or (nd == 3 and xyz.shape[1] != 3):
        raise ValueError('Incorrect number of coordinates (%d) for %dD DLT (it should be %d).' %(xyz.shape[1],nd,nd))
    if nd == 3 and np < 6 or nd == 2 and np < 4:
        raise ValueError('%dD DLT requires at least %d calibration points. Only %d points were entered.' %(nd, 2*nd, np))
        
    #Normalize the data to improve the DLT quality (DLT is dependent of the system of coordinates).
    #This is relevant when there is a considerable perspective distortion.
    #Normalization: mean position at origin and mean distance equals to 1 at each direction.
    Txyz, xyzn = Normalization(nd, xyz)
    Tuv, uvn = Normalization(2, uv)

    A = []
    if nd == 2: #2D DLT
        for i in range(np):
            x,y = xyzn[i,0], xyzn[i,1]
            u,v = uvn[i,0], uvn[i,1]
            A.append( [x, y, 1, 0, 0, 0, -u*x, -u*y, -u] )
            A.append( [0, 0, 0, x, y, 1, -v*x, -v*y, -v] )
    elif nd == 3: #3D DLT
        for i in range(np):
            x,y,z = xyzn[i,0], xyzn[i,1], xyzn[i,2]
            u,v = uvn[i,0], uvn[i,1]
            A.append( [x, y, z, 1, 0, 0, 0, 0, -u*x, -u*y, -u*z, -u] )
            A.append( [0, 0, 0, 0, x, y, z, 1, -v*x, -v*y, -v*z, -v] )

    #convert A to array
    A = N.asarray(A) 
    #Find the 11 (or 8 for 2D DLT) parameters:
    U, S, Vh = N.linalg.svd(A)
    #The parameters are in the last line of Vh and normalize them:
    L = Vh[-1,:] / Vh[-1,-1]
    #Camera projection matrix:
    H = L.reshape(3,nd+1)
    #Denormalization:
    H = N.dot( N.dot( N.linalg.pinv(Tuv), H ), Txyz );
    H = H / H[-1,-1]
    L = H.flatten()
    #Mean error of the DLT (mean residual of the DLT transformation in units of camera coordinates):
    uv2 = N.dot( H, N.concatenate( (xyz.T, N.ones((1,xyz.shape[0]))) ) ) 
    uv2 = uv2/uv2[2,:] 
    #mean distance:
    err = N.sqrt( N.mean(N.sum( (uv2[0:2,:].T - uv)**2,1 )) ) 

    return L, err

In [86]:
M, err1 = DLTcalib(2,src2, dst2)

test1
test1


In [87]:
M.reshape(3,3)

array([[ 1.738136e+00,  9.499032e-03, -4.466956e+02],
       [ 2.745448e-01,  1.514703e+00, -1.213725e+02],
       [ 1.160021e-03,  2.073185e-05,  1.000000e+00]])

In [83]:
M.reshape(3,3)

array([[ 1.738142e+00,  9.499591e-03, -4.466981e+02],
       [ 2.745430e-01,  1.514708e+00, -1.213726e+02],
       [ 1.160019e-03,  2.073913e-05,  1.000000e+00]])

In [79]:
M.reshape(3,3)

array([[ 1.738149e+00,  9.498246e-03, -4.466993e+02],
       [ 2.745501e-01,  1.514710e+00, -1.213745e+02],
       [ 1.160043e-03,  2.072784e-05,  1.000000e+00]])